In [1]:
import pandas as pd
import elasticsearch
es = elasticsearch.Elasticsearch('192.168.200.1')

In [2]:
def process_es_search(flows):
    return [[x['_source']['nuage_metadata']['service'],
            x['_source']['nuage_metadata']['sourcevport'],
            x['_source']['nuage_metadata']['acl_source_name']] for x in flows]
    
data = []
page = es.search(index='nuage_flow_test',
                 doc_type='nuage_doc_type',
                 body={"query": {"match_all": {}}},
                scroll = '2m',
                size = 10000)

#print(page['hits']['hits'][0])
data += process_es_search(page['hits']['hits'])
sid = page['_scroll_id']
scroll_size = len(page['hits']['hits'])
while scroll_size>0:
    page = es.scroll(scroll_id = sid, scroll = '2m')
    data += process_es_search(page['hits']['hits'])
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])

print(data[0])



['Kerberos', 'PG4-VPort2', 'PG4']


In [3]:
from pprint import pprint
pprint(data[0])
print(len(data))

['Kerberos', 'PG4-VPort2', 'PG4']
360000


In [4]:
cols = ['service','sourcevport','acl_source_name']
df = pd.DataFrame(data,columns=cols)
df.head()

,service,sourcevport,acl_source_name
0,Kerberos,PG4-VPort2,PG4
1,Kerberos,PG4-VPort2,PG4
2,Kerberos,PG4-VPort2,PG4
3,Kerberos,PG4-VPort2,PG4
4,Kerberos,PG4-VPort2,PG4


In [5]:
df.memory_usage()

Index                   80
service            2880000
sourcevport        2880000
acl_source_name    2880000
dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder
import pickle
import xgboost
xgb = xgboost.XGBClassifier(gamma = 0.1)


le = LabelEncoder()
le.fit(df['service'])
df['service'] = le.transform(df['service'])

le2 = LabelEncoder()
le2.fit(df['sourcevport'])
df['sourcevport'] = le2.transform(df['sourcevport'])

le3 = LabelEncoder()
le3.fit(df['acl_source_name'])
df['acl_source_name'] = le3.transform(df['acl_source_name'])

X = df[['service','sourcevport']] #Predictor variable
Y = df['acl_source_name'] #Response variable


/home/vagrant/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
import numpy as np
np.random.seed(123)
msk = np.random.randn(len(df))<0.75

In [8]:
trainX,testX = X[msk],X[~msk]
trainY,testY = Y[msk],Y[~msk]

In [9]:
print(trainX.shape,trainY.shape,testX.shape,testY.shape)

(278186, 2) (278186,) (81814, 2) (81814,)


In [10]:
model = xgb.fit(trainX,trainY)

In [11]:
predictedY = model.predict(testX)

In [12]:
from sklearn.metrics import recall_score,precision_score

In [13]:
print("Recall = ",recall_score(testY,predictedY,average='micro'),'\n')
print("Precision = ",precision_score(testY,predictedY,average='micro'),'\n')

Recall =  1.0 

Precision =  1.0 

